# Description

Creates a point of comparison with non-optimized version of clustermatch.

# Remove pycache dir

In [1]:
!echo ${CODE_DIR}

/opt/code


In [2]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

/opt/code/libs/clustermatch/__pycache__


In [3]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -exec rm -rf {} \;

find: ‘/opt/code/libs/clustermatch/__pycache__’: No such file or directory


In [4]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

# Modules

In [5]:
import numpy as np

from clustermatch.coef import _cm

# Data

In [6]:
N_REPS = 100
N_SAMPLES = 10000

In [7]:
np.random.seed(0)

In [8]:
x = np.random.rand(N_SAMPLES)
y = np.random.rand(N_SAMPLES)

# Run

In [9]:
def func():
    for i in range(N_REPS):
        # py_func accesses the original python function, not the numba-optimized one
        # this is needed to be able to profile the function
        _cm.py_func(x, y)

In [10]:
%%timeit func()
func()

18.3 s ± 722 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%prun -s cumulative -l 20 -T 00-previous_version.txt
func()

 
*** Profile printout saved to text file '00-previous_version.txt'. 


         1504 function calls in 17.720 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   17.720   17.720 {built-in method builtins.exec}
        1    0.000    0.000   17.720   17.720 <string>:1(<module>)
        1    0.001    0.001   17.720   17.720 1556911885.py:1(func)
      100    0.015    0.000   17.719    0.177 coef.py:266(_cm)
      200    9.687    0.048    9.687    0.048 coef.py:160(_get_parts)
      100    8.013    0.080    8.013    0.080 coef.py:199(cdist_parts)
      200    0.001    0.000    0.001    0.000 coef.py:118(_get_range_n_clusters)
      200    0.001    0.000    0.001    0.000 {built-in method numpy.zeros}
      100    0.001    0.000    0.001    0.000 {method 'argmax' of 'numpy.ndarray' objects}
      100    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
      100    0.000    0.000    0.000    0.000 coef.py:249(unravel_index_2d)
      100    0.000    0.000    

The bottleneck functions are, in order of importance:
1. `_get_parts`
1. `cdist_parts`